In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

/opt/conda/envs/rerank/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# t5model = T5ForConditionalGeneration.from_pretrained('bigscience/T0_3B', torch_dtype=torch.float32).cuda()
t5model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-large', torch_dtype=torch.float32).cuda()
device = torch.device('cuda')
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')

In [3]:
# device = torch.device('cuda')
# tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')

Downloading: 100%|██████████| 792k/792k [00:00<00:00, 38.3MB/s]
Downloading: 100%|██████████| 2.20k/2.20k [00:00<00:00, 3.40MB/s]
Downloading: 100%|██████████| 2.54k/2.54k [00:00<00:00, 3.90MB/s]


In [4]:
for param in t5model.parameters():
    param.requires_grad = False
    
t5model = t5model.to(device)

t5model = t5model.eval()

In [5]:
def get_answer(prompt):
    input_encoding = tokenizer(prompt ,padding='longest',max_length=512,pad_to_multiple_of=8,truncation=True,return_tensors='pt').to(device)
    print(tokenizer(prompt ,return_tensors='pt').to(device).input_ids.shape)
    print(input_encoding.input_ids.shape)
    return tokenizer.decode(t5model.generate(input_encoding.input_ids)[0], skip_special_tokens=True)

In [20]:
# text = '''Import the certification file into the agent by doing the following command:

# Note : You can use either .cer or .pem.
# Make sure the file is in ASCII not binary.
# Where the /usr/ITM/JRE is your Java location and <fileName> is your file name.'''

# text = 'A CSV document is basically a plain text file. If the mime type of the document as shown in system properties in ICN is not text/csv, then Navigator cannot recognize the document and a .dat extension will be added upon download. The correct mime type for a CSV file is text/csv. '
# text = 'The transaction timeout for RTS/DC is set to 10 minutes by default in the teamserver web module deployment descriptor (WEB-INF/web.xml), and application servers also have container level transaction timeouts that are configurable. If a transactional operation performed by RTS/DC takes longer than these timeouts to complete, the transaction is rolled back and the operation is not completed.'
# text = '''

# During web server start-up, errors indicating failures are logged in the plug-in log file (http_plugin.log):

# ERROR: lib_security: loadSecurityLibrary: Failed to load gsk library from opt/IBM/WebSphere/Plugins/gsk8/gsk8_64
# ERROR: ws_transport: transportInitializeSecurity: Unable to load security library
# ERROR: ws_server: serverAddTransport: Failed to initialize security
# ERROR: ws_server: serverAddTransport: HTTPS Transport is skipped

# If an older level of the GSkit library is present in the /usr directory, plug-in connection handshake failures occur with GSK_ERROR_BAD_PEER related messages.
# '''
# prompt = f'Passage: {text}. Please generate a how-to problem statement for this passage.'
# prompt = f'Cause: {text}. Please generate a problem statement for the above cause.'
example1 = 'Passage: CVEID:CVE-2017-14919 [http://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2017-14919]DESCRIPTION:Node.js is vulnerable to a denial of service, caused by an uncaught exception flaw in the zlib module. Query: Help with Security Bulletin: API Connect is affected by a Node.js denial of service vulnerability (CVE-2017-14919). Where can I find this information?'
example2 = 'Passage: If an issue arises during DW trace persistence at runtime, the related messages and stack traces are only available programmatically through the following API: ilog.rules.res.session.IlrSessionResponse.getWarnings() So it is possible for an error to remain silent if the warnings above are not explicitly logged by the application. Query: No trace is logged in Decision Warehouse, and no records in EXECUTION_TRACES table for a certain ruleset. How can I debug this issue?'
# # prompt = f'Please generate a how-to problem statement for this passage. Below are some examples separated by xxxxx: \n {example1} xxxxx \n {example2}\n. Passage: {text}. Question:'
text = 'When the complete path to the silent response file is not specified as part of the command invoking the silent install, it fails with the following error message in the log file'
text = 'If silent unisntall fails, then reinstall'
question = 'Silent uninstall of ITCAM for MQ agent fails'
prompt = f'Passage: {text}. Question: {question}. Does the question and answer have overlapping data?'
# prompt = '''Passage: CVEID:CVE-2017-14919 [http://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2017-14919]DESCRIPTION:Node.js is vulnerable to a denial of service, caused by an uncaught exception flaw in the zlib module. Query: Help with Security Bulletin: API Connect is affected by a Node.js denial of service vulnerability (CVE-2017-14919). Where can I find this information?'
# Passage: If an issue arises during DW trace persistence at runtime, the related messages and stack traces are only available programmatically through the following API: ilog.rules.res.session.IlrSessionResponse.getWarnings() So it is possible for an error to remain silent if the warnings above are not explicitly logged by the application. Query: No trace is logged in Decision Warehouse, and no records in EXECUTION_TRACES table for a certain ruleset. How can I debug this issue?
# Passage: When the complete path to the silent response file is not specified as part of the command invoking the silent install, it fails with the following error message in the log file. Query:'''

get_answer(prompt)

torch.Size([1, 43])
torch.Size([1, 48])


'no'

In [9]:
prompt

'Passage: When the complete path to the silent response file is not specified as part of the command invoking the silent install, it fails with the following error message in the log file. Question: Silent uninstall of ITCAM for MQ agent fails. Does the passage answer the question?'